# Notes: `Understanding Python Metaclasses`

Em python tudo é um objeto (tudo mesmo) até mesmo as classes criadas para criar novas instâncias são objetos. As classes então, são instâncias de algo chamado `metaclass`. Em python a metaclass default da quais as classes são instâncias é a `type`. Isso é um pouco confuso pois type também é a built-in function que retorna o tipo de um objeto.

In [1]:
class Student:
    pass

In [2]:
# Aqui podemos ver que a classe em si é uma instância da metaclass type

type(Student)

type

In [5]:
# E analisando a instância da classe Student, a função type retorna que o tipo é o __main__.Student

student = Student()

type(student)

__main__.Student

Ou da mesma forma podemos fazer a análise utilizando a função built-in `isinstance`

In [7]:
print(isinstance(Student, type))
print(isinstance(student, Student))

True
True


Verifico assim a existência de uma metaclasse responsável por criar as classes em sí.

# Diagrama de criação de instâncias e classes

![](./src/diagram_instances_class_metaclass.jpg)

### Criando uma classe a partir do built-in type.

In [9]:
MyClass = type('MyClass', (), {})
MyClass

__main__.MyClass

Reparo que consigo criar uma classe sem utilizar a keyword `class`.

Já tinha visto em outras fontes essa forma de criação de classes.

- Livro sobre design patterns (falando sobre singleton)
- Outros artigos. 

Nesta forma de criação, a tuple posicionada no segundo argumento cuida da questão hierarquica, definindo de quais classes a classe criada irá herdar. O dicionário presente no terceiro argumento representa a definição dos atributos.

In [11]:
# Outro exemplo

School = type('School', tuple([object]), {'name': 'Colégio Mercúrio', 'address': 'Rua Mercúrio, 179'})

In [18]:
School.__base__

object

In [19]:
print(School.name)
print(School.address)

Colégio Mercúrio
Rua Mercúrio, 179


### Questionamento:

quando passo os atributos dessa forma através da built-in type, eu na verdade estou criando atributos da classe compartilhado entre as instâncias? `Boa pergunta!`

# Criando uma `metaclass` customizada:

In [39]:
class Meta(type):
    
    def __call__(self):
        
        print("teste chamada de metaclasse")

class Complex(metaclass=Meta):
    
    pass

print(type(Complex))

Complex()

<class '__main__.Meta'>
teste chamada de metaclasse


Consigo então, criar uma <font color='green'>metaclass</font> customizada baseada no tipo default `type` e posso utiliza-lo para ser a <font color='green'>metaclass</font> de refência para instância de outras classes.
Outro teste que lembrei de fazer foi observar que a criação de uma instância de uma classe (objeto convencional), chama o `__call__` da metaclass. Com isso consigo entender porque esta pode ser um método utilizado em `singleton`, colocando como responsabilidade de uma <font color='green'>metaclass</font> o controle sobre o número de instâncias criadas. 

O artigo comenta que a palavra-chave <font color='blue'>class</font> não é somente uma abstração de linguagem para facilitar a utilização (syntatic sugar) mas que é mais que isso, a chamada dessa forma faz algumas coisas extras como chamar alguns dunder methods que ajudam na criação e definição da classe.

# Magical methods